# Baseline 알고리즘 기반 추천시스템

## #01. 준비작업

### [1] 패키지 가져오기

In [39]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from helper.util import *
from helper.plot import *
from helper.analysis import *
from helper.classification import *

from surprise import Reader, Dataset, BaselineOnly
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from surprise.accuracy import rmse, mae

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### [2] 데이터 가져오기

In [40]:
origin_total = my_read_excel("https://data.hossam.kr/mldata/movie_ratings.xlsx", sheet_name=[0,1,2,3], info=False)

In [78]:
origin = origin_total[0]
origin

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## #02. 데이터 전처리

### [1] Surprise 형식의 데이터로 변환

`사용자 번호, 아이템 번호, 평점` 구조의 데이터를 만족해야 한다.

In [42]:
df = origin.drop('timestamp', axis=1)

# 평점의 분포를 알려준다.
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df, reader)
data

## #03. 추천 모형 구현

### [1] 기본 코드

#### (1) 훈련, 검증 데이터 분리

sklearn이 아닌 surprise 자체 함수 사용

In [43]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

#### (2) 추천 모형 학습 후 성능 평가

In [44]:
estimator = BaselineOnly()
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Estimating biases using als...


[Prediction(uid=603, iid=3996, r_ui=5.0, est=3.865470094018238, details={'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=3.5270186068257785, details={'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.2531312891488335, details={'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=4.143871102075767, details={'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=2.6344308636371943, details={'was_impossible': False})]

> uid: 사용자 번호, iid: 아이템 번호, r_ui: 해당 사용자가 실제로 부여한 평점, est: 예측평점

#### (3) 특정 유저가 특정 영화에 부여할 평점 예상

In [45]:
upred = estimator.predict(uid=603, iid=3996)
upred

Prediction(uid=603, iid=3996, r_ui=None, est=3.865470094018238, details={'was_impossible': False})

In [46]:
upred.est

3.865470094018238

#### (4) 성능평가

In [47]:
rmse(pred), mae(pred)

RMSE: 0.8715
MAE:  0.6706


(0.8715309792778995, 0.6706040327595953)

### [2] 교차검증

#### (1) 교차검증을 위한 하이퍼파라미터 설정

In [48]:
estimator = BaselineOnly(bsl_options={
    "method": "als",    # 알고리즘 "als" or "sgd"
    "n_epochs": 10,     # 반복횟수 (기본값=10)
    "reg_u": 10,        # 항목에 대한 정규화 매개변수 (기본값=10)
    "reg_i": 15         # 사용자를 위한 정규화 매개변수 (기본값=15)
})
cv_result = cross_validate(estimator, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8837  0.8719  0.8790  0.8741  0.8736  0.8765  0.0043  
MAE (testset)     0.6820  0.6734  0.6785  0.6730  0.6753  0.6765  0.0034  
Fit time          0.19    0.24    0.22    0.20    0.20    0.21    0.02    
Test time         0.19    0.04    0.03    0.05    0.04    0.07    0.06    


#### (2) 교차검증 결과 확인

In [49]:
cv_result

{'test_rmse': array([0.88373637, 0.87193512, 0.87902998, 0.87407299, 0.87364415]),
 'test_mae': array([0.6820075 , 0.67342862, 0.67845291, 0.67303274, 0.67533935]),
 'fit_time': (0.18988370895385742,
  0.24279356002807617,
  0.2168574333190918,
  0.19869565963745117,
  0.20479846000671387),
 'test_time': (0.19308805465698242,
  0.04491591453552246,
  0.034789085388183594,
  0.04957723617553711,
  0.043417930603027344)}

#### (3) 교차검증 성능 평가 지표 출력

In [50]:
print("RMSE(mean):", cv_result['test_rmse'].mean())
print("MAE(mean):", cv_result['test_mae'].mean())

RMSE(mean): 0.8764837237103705
MAE(mean): 0.6764522246398658


### [3] 하이퍼파라미터 튜닝

#### (1) 학습 모형 구성

In [51]:
params = {
    'bsl_options': {
        "method": ["als", "sgd"],    # 알고리즘 "als" or "sgd"
        "n_epochs": [10, 20],        # 반복횟수 (기본값=10)
        "reg_u": [10, 12],           # 사용자에 대한 정규화 매개변수 (기본값=10)
        "reg_i": [15, 20]            # 아이템에 대한 정규화 매개변수 (기본값=15)
    }
}

# grid = GridSearchCV(BaselineOnly, 
#                     param_grid=params, 
#                     measures=['RMSE', 'MAE'], 
#                     cv=5, 
#                     n_jobs=-1)
                    
grid = RandomizedSearchCV(BaselineOnly, 
                        param_distributions=params, 
                        measures=['RMSE', 'MAE'], 
                        cv=5, 
                        n_jobs=-1, 
                        random_state=1234)

grid.fit(data)

#### (2) 성능 평가 지표 확인

In [52]:
grid.best_score

{'rmse': 0.8697938092185622, 'mae': 0.6688372599876551}

#### (3) 최적 하이퍼파라미터 확인

In [53]:
grid.best_params

{'rmse': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 12,
   'reg_i': 20}},
 'mae': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 12,
   'reg_i': 20}}}

#### (4) 최적 추정기

In [54]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x202184b7910>,
 'mae': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x202184b7510>}